In [28]:
import random

# Límites
lsup = 10
linf = 0
linf_lp = 3
linf_ss = 2
iteraciones = 50  # Límite de iteraciones
lim = 5  # Límite de evaluaciones
nobreras = 20  # Número de obreras

peso_max = 30  # Límite de peso de la mochila
aptitudes = [0] * nobreras
costos = [0] * nobreras
abejas = []
lim_intentos = [0] * nobreras


# Población inicial aleatoria
def poblacion_inicial(linf, lsup):
    return round(linf + random.uniform(0, 1) * (lsup - linf))

def aptitud(individuo):
    return sum([
        individuo[0] * 10, 
        individuo[1] * 8, 
        individuo[2] * 12, 
        individuo[3] * 6, 
        individuo[4] * 3, 
        individuo[5] * 2, 
        individuo[6] * 2
    ])

def restriccion(individuo):
    return sum([
        individuo[0] * 4,
        individuo[1] * 2,
        individuo[2] * 5,
        individuo[3] * 5,
        individuo[4] * 2,
        individuo[5] * 1.5,
        individuo[6] * 1
    ])

def inicializa_abeja(indice):
    abeja = []
    
    # Coordenadas de la abeja
    abeja.append(poblacion_inicial(linf, lsup))  # sol 1
    abeja.append(poblacion_inicial(linf_lp, lsup))  # sol 2
    abeja.append(poblacion_inicial(linf_ss, lsup))  # sol 3
    for j in range(4):
        abeja.append(poblacion_inicial(linf, lsup))  # sol 4-7
    
    # Costo de la abeja
    costos[indice] = restriccion(abeja)
    
    # Asegurar que la solución sea posible
    while costos[indice] >= peso_max:
        abeja = []
        abeja.append(poblacion_inicial(linf, lsup))  # sol 1
        abeja.append(poblacion_inicial(linf_lp, lsup))  # sol 2
        abeja.append(poblacion_inicial(linf_ss, lsup))  # sol 3
        for j in range(4):
            abeja.append(poblacion_inicial(linf, lsup))  # sol 4-7
        
        costos[indice] = restriccion(abeja)
    
    # Aptitud de la abeja
    aptitudes[indice] = aptitud(abeja)
    
    print(abeja)
    return abeja

def fase_inicio():
    # Inicializamos la colonia de abejas
    for i in range(nobreras):
        abeja_aux = inicializa_abeja(i)
        abejas.append(abeja_aux)

def actualizar_abeja(indice):
    aux = []
    abeja_random = random.randint(0, nobreras - 1)
    while indice == abeja_random:
        abeja_random = random.randint(0, nobreras - 1)
    x = abejas[indice]  # Abeja actual
    k = abejas[abeja_random]  # Abeja aleatoria
    j = random.randint(0, 6)  # Posición aleatoria
    #print(f"Posición modificada: {j+1}, con la abeja número: {abeja_random+1}")
    vmodif = round(x[j] + random.uniform(-1, 1) * (x[j] - k[j]))
    if j == 1:
        while vmodif < linf_lp:
            vmodif = round(x[j] + random.uniform(-1, 1) * (x[j] - k[j]))  
        #print("hace falta lp")
    if j == 2:
        while vmodif < linf_ss:
            vmodif = round(x[j] + random.uniform(-1, 1) * (x[j] - k[j])) 
        #print("hace falta ss") 
    if vmodif < linf:
        vmodif=linf
    if vmodif > lsup:
        vmodif=lsup


    aux = x
    aux[j] = vmodif
    return aux 

        
def fase_obrera():
    # Generamos una nueva solución
    cambio_posible = [None] * nobreras  # vector auxiliar

    # Empezamos a ciclar actualizaciones
    for i in range(nobreras):
        # Encontrar posible mejora a la solucion
        cambio_posible[i] = actualizar_abeja(i)

        # Si la abeja es muy pesada busca otra solucion sin aumentatr el limite
        while restriccion(cambio_posible[i]) > 30:
            cambio_posible[i] = actualizar_abeja(i)

        # Medir la aptitud del posible cambio para actualizar el vector y los limites
        apt = aptitud(cambio_posible[i])
        if apt > aptitudes[i]:
            abejas[i] = cambio_posible[i]
            aptitudes[i] = apt
            costos[i] = restriccion(cambio_posible[i])
            lim_intentos[i] = 0
            print("si actualizo")
        else:
            if lim_intentos[i] is None:
                lim_intentos[i] = 0  # Initialize if accidentally None
            lim_intentos[i] += 1
            print("no actualizo")
        print(abejas[i])

    cambio_posible.clear()

def probabilidad_acumulada(aptitudes):
    denum = sum(aptitudes)
    aux = 0
    acumulada = [0]*nobreras
    for i in range(nobreras):
        aux += aptitudes[i] / denum
        acumulada[i] = aux
    return acumulada

def ruleta(pac):
    r1 = random.uniform(0, 1)

    for i in range(nobreras):
        if r1 <= pac[i]:  # Cambia la condición a <=
            return i

    # Si no se encuentra ningún índice (por seguridad), retorna el último
    return nobreras - 1



def fase_observadora():
    pac = [0] * nobreras
    pac = probabilidad_acumulada(aptitudes)
    print(pac)

    cambio_posible2 = [None] * nobreras # vector auxiliar

    for i in range(nobreras):
        x = ruleta(pac)

        cambio_posible2[i]  = actualizar_abeja(i)

        # Si la abeja es muy pesada busca otra solucion sin aumentatr el limite
        while restriccion(cambio_posible2[i]) >30 : 
            cambio_posible2[i] = actualizar_abeja(i)
           # print("estoy gordote")

        # Medir la aptitud del posible cambio para actualizar el vector y los limites
        apt = aptitud(cambio_posible2[i])
        if apt > aptitud(abejas[x]):
            abejas[x] = cambio_posible2[i]
            aptitudes[x] = apt
            costos [x] = restriccion(cambio_posible2[i])
            lim_intentos[x] = 0
            print("si actualizo")
        else:
            lim_intentos[x]=lim_intentos[x]+1
            print("no actualizo")
        print(abejas[x])
    
    cambio_posible2.clear()
    pac.clear()

def fase_limites():
    for i in range(nobreras):
        if lim_intentos[i] > lim:
            abejas[i] = inicializa_abeja(i)

    
for i in range(iteraciones):
    # Ejecución
    if i == 0:
        print("\n\n\niniciando")
        fase_inicio()
        print("\n\n\nfase obrera")
        fase_obrera()
        print("\n\n\nfase observadora ")
        fase_observadora()
        print("\n\n\nfase re-exploracion ")
        fase_limites()
    else:
        print("\n\n\nfase obrera")
        fase_obrera()
        print("\n\n\nfase observadora ")
        fase_observadora()
        print("\n\n\nfase re-exploracion ")
        fase_limites()

   
print("Resultado Final")
best_index = aptitudes.index(max(aptitudes))
print(abejas[best_index])  # Ahora imprime la abeja con la mayor aptitud
print(max(aptitudes))
print(costos[best_index])






iniciando
[1, 4, 3, 0, 0, 0, 2]
[1, 4, 2, 1, 0, 0, 1]
[1, 4, 2, 0, 2, 0, 1]
[0, 5, 2, 0, 0, 3, 3]
[2, 3, 2, 0, 0, 2, 1]
[0, 6, 2, 0, 1, 1, 3]
[0, 4, 3, 0, 1, 1, 3]
[0, 8, 2, 0, 0, 2, 0]
[1, 3, 2, 0, 3, 0, 1]
[1, 3, 2, 1, 0, 2, 0]
[2, 4, 2, 0, 0, 0, 3]
[0, 4, 3, 0, 1, 1, 1]
[0, 5, 2, 0, 1, 0, 7]
[1, 3, 2, 0, 2, 0, 3]
[0, 3, 2, 0, 1, 1, 6]
[1, 4, 2, 0, 0, 1, 5]
[0, 3, 2, 2, 0, 1, 1]
[0, 3, 3, 0, 1, 0, 3]
[0, 4, 2, 0, 1, 4, 1]
[1, 4, 3, 0, 0, 1, 1]



fase obrera
no actualizo
[1, 4, 3, 0, 0, 0, 1]
no actualizo
[1, 4, 2, 1, 0, 0, 1]
no actualizo
[1, 4, 2, 0, 2, 0, 1]
no actualizo
[0, 5, 2, 0, 0, 3, 3]
no actualizo
[2, 3, 2, 0, 0, 2, 1]
no actualizo
[0, 6, 2, 0, 1, 1, 3]
no actualizo
[0, 4, 3, 0, 1, 1, 3]
no actualizo
[0, 8, 2, 0, 0, 2, 0]
no actualizo
[1, 3, 2, 0, 2, 0, 1]
no actualizo
[1, 3, 2, 1, 0, 2, 0]
no actualizo
[2, 4, 2, 0, 0, 0, 3]
no actualizo
[0, 4, 3, 0, 1, 1, 1]
no actualizo
[0, 5, 2, 0, 1, 0, 7]
si actualizo
[1, 3, 2, 0, 2, 1, 3]
no actualizo
[0, 3, 2, 0, 1, 1, 6]
no actu